In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu

import tensorflow as tf
from latent.ml.dl_subclass import MLP, MLPbeta, CNN, eval_nn, get_train
import latent.session_subclass as session
import latent.utils.data_utils as prd
from latent.ml.lda import train_lda, eval_lda

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'TR'
with open('traindata/train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

In [5]:
# Set session params
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'tdar','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven14'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = np.max(params[:,0])

data_load = False
test = False
train = True

if test and not data_load:
    with open('real_noise/all_real_noise.p', 'rb') as f:
        real_noise_temp, _ = pickle.load(f)

clean_all = np.zeros((n_subs,8))
noisy_all = np.zeros((n_subs,8))

for sub in range(1,n_subs+1):
    print('Running ' + str(sess.sub_type) + str(sub))
    sess.sub = sub
    tf.keras.backend.clear_session()

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

    if np.sum(train_ind):# and np.sum(test_ind):
        n_dof = np.max(params[train_ind,4])
    
        # Train NNs
        mlp = MLP(n_class=n_dof, latent_dim=8)
        cnn = CNN(n_class=n_dof, latent_dim=8)

        optimizer = tf.keras.optimizers.Adam()
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        if train:
            if data_load:
                try:
                    with open('subclass/train/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p', 'rb') as f:
                        x_train_cnn, x_train_mlp, x_train_lda, x_train_aug, y_train, y_train_lda, sess.scaler = pickle.load(f)
                except:
                    print('no training data to load')
                    data_load = False
                
                trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
                traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
            
            if not data_load:   
                trainmlp, _, traincnn, _, y_train, _, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug = prd.prep_train_data(sess,raw,params)

            # Train neural networks
            models = [mlp, cnn]
            for model in models:
                if isinstance(model,CNN):
                    ds = traincnn
                    del trainmlp
                else:
                    ds = trainmlp
                
                train_mod = get_train()

                for epoch in range(sess.epochs):
                    # Reset the metrics at the start of the next epoch
                    train_loss.reset_states()
                    train_accuracy.reset_states()

                    for x, y in ds:
                        train_mod(x, y, model, optimizer, train_loss, train_accuracy)

                    if epoch == 0 or epoch == sess.epochs-1:
                        print(
                            f'Epoch {epoch + 1}, '
                            f'Loss: {train_loss.result():.2f}, '
                            f'Accuracy: {train_accuracy.result() * 100:.2f} '
                        )
                tf.keras.backend.clear_session()
                
                del train_mod

            # Train aligned LDA
            y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

            mlp_enc = mlp.get_layer(name='enc')
            w_mlp, c_mlp,_, _, v_mlp = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

            del x_train_mlp
            cnn_enc = cnn.get_layer(name='enc')
            temp2 = np.vstack((cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//4,...]),cnn_enc(x_train_cnn[x_train_cnn.shape[0]//4:x_train_cnn.shape[0]//2,...]).numpy()))
            temp2 = np.vstack((temp2, cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:3*x_train_cnn.shape[0]//4,...]),cnn_enc(x_train_cnn[3*x_train_cnn.shape[0]//4:,...]).numpy()))
            w_cnn, c_cnn,_, _, v_cnn = train_lda(temp2,y_train_aug)

            # Train LDA
            w,c, mu_class, C, v = train_lda(x_train_lda,y_train_lda)
            w_aug,c_aug, _, _, v_noise = train_lda(x_train_aug,y_train_aug)

            mlp_w = mlp.get_weights()
            cnn_w = cnn.get_weights()
            mlpb_w = 0
            w_mlpbeta=0
            c_mlpbeta=0
            with open('subclass/models_apple/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '_red.p', 'wb') as f:
                pickle.dump([v_mlp, v_mlp, v_cnn, v_cnn, v_cnn, v, v_noise],f)
            with open('subclass/models_apple/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p','wb') as f:
                pickle.dump([mlp_w, cnn_w, w_mlp, c_mlp, w_cnn, c_cnn, w, c, w_aug, c_aug, sess.emg_scale, sess.scaler, mu_class,C],f)
        else:
            with open('subclass/models_apple/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p','rb') as f:
                mlp_w, cnn_w, w_mlp, c_mlp, w_cnn, c_cnn, w, c, w_aug, c_aug, sess.emg_scale, sess.scaler, mu_class, C = pickle.load(f)

            mlp_enc = mlp.get_layer(name='enc')
            cnn_enc = cnn.get_layer(name='enc')

        if test:
            # Load test data
            if data_load:
                try:
                    with open('subclass/test/' + str(sess.sub_type) + str(sess.sub) + '_' + str(sess.feat_type) + '_' + str(sess.test) + '.p', 'rb') as f:
                        x_test_lda, y_test, clean_size = pickle.load(f)
                except:
                    print('no testing data to load')
                    data_load = False

                x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],int(x_test_lda.shape[1]/6),-1)),(0,2,1))[...,np.newaxis]
                x_test_cnn = sess.scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
                x_test_cnn = x_test_cnn.astype('float32')

                # Reshape for nonconvolutional SAE
                x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
            if not data_load:
                x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)

                with open('subclass/test/' + str(sess.sub_type) + str(sess.sub) + '_' + str(sess.feat_type) + '_' + str(sess.test) + '.p', 'wb') as f:
                    pickle.dump([x_test_lda, y_test, clean_size],f)

            # workaround
            mlp(x_test_mlp[:2,...])
            cnn(x_test_cnn[:2,...])

            mlp.set_weights(mlp_w)
            cnn.set_weights(cnn_w)
        
            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            # LDA
            clean_lda, noisy_lda = eval_lda(w, c, x_test_lda, y_test_aligned, clean_size)
            # AUG-LDA
            clean_aug, noisy_aug = eval_lda(w_aug, c_aug, x_test_lda, y_test_aligned, clean_size)
            # MLP
            clean_mlp, noisy_mlp = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_mlplda, noisy_mlplda = eval_lda(w_mlp, c_mlp, mlp_test_aligned, y_test_aligned, clean_size)
            # CNN
            clean_cnn, noisy_cnn = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_cnnlda, noisy_cnnlda = eval_lda(w_cnn, c_cnn, cnn_test_aligned, y_test_aligned, clean_size)

            print( 
                f'LDA ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
                f'\nAUG ---- '
                f'Clean: {clean_aug * 100:.2f}, '
                f'Noisy: {noisy_aug * 100:.2f}'
                f'\nMLP ---- '
                f'Clean: {clean_mlp * 100:.2f}, '
                f'Noisy: {noisy_mlp * 100:.2f}, '
                f'LDA Clean: {clean_mlplda * 100:.2f}, '
                f'LDA Noisy: {noisy_mlplda * 100:.2f}'
                f'\nCNN ---- '
                f'Clean: {clean_cnn * 100:.2f}, '
                f'Noisy: {noisy_cnn * 100:.2f}, '
                f'LDA Clean: {clean_cnnlda * 100:.2f}, '
                f'LDA Noisy: {noisy_cnnlda * 100:.2f}'
            )

            clean_all[sub-1,:] = np.stack((clean_lda,clean_aug,clean_mlp,clean_mlplda,clean_cnn,clean_cnnlda))
            noisy_all[sub-1,:] = np.stack((noisy_lda,noisy_aug,noisy_mlp,noisy_mlplda,noisy_cnn,noisy_cnnlda))
    else:
        print('no training or testing data')

Running TR1
Loading training data: traindata_all/TR1_traindata_2.p
Epoch 1, Loss: 1.74, Accuracy: 31.67 
Epoch 30, Loss: 0.72, Accuracy: 72.33 
Epoch 1, Loss: 0.88, Accuracy: 68.05 
Epoch 30, Loss: 0.20, Accuracy: 92.40 
Running TR2
Loading training data: traindata_all/TR2_traindata_2.p
Epoch 1, Loss: 1.78, Accuracy: 29.67 
Epoch 30, Loss: 0.56, Accuracy: 77.60 
Epoch 1, Loss: 0.72, Accuracy: 74.21 
Epoch 30, Loss: 0.12, Accuracy: 95.20 
Running TR3
Loading training data: traindata_all/TR3_traindata_2.p
Epoch 1, Loss: 1.63, Accuracy: 28.84 
Epoch 30, Loss: 0.80, Accuracy: 68.10 
Epoch 1, Loss: 0.99, Accuracy: 60.61 
Epoch 30, Loss: 0.23, Accuracy: 91.33 
Running TR4
Loading training data: traindata_all/TR4_traindata_2.p
Epoch 1, Loss: 1.44, Accuracy: 38.13 
Epoch 30, Loss: 0.47, Accuracy: 78.48 
Epoch 1, Loss: 0.63, Accuracy: 73.47 
Epoch 30, Loss: 0.15, Accuracy: 94.00 
Running TR5
Loading training data: traindata_all/TR5_traindata_2.p
Epoch 1, Loss: 1.35, Accuracy: 41.37 
Epoch 30, L

Plotting

In [3]:
# Plot example signals
import matplotlib.pyplot as plt
import copy as cp
plt.rcParams['figure.dpi'] = 300
%matplotlib qt

In [20]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven14'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
x_noise,x,emgscale =prd.prep_noise_data(sess,raw,params)
with open('real_noise/all_real_noise.p', 'rb') as f:
    real_noise_temp, _ = pickle.load(f)

i = 3000

# Plot original example signal
fig,ax = plt.subplots(2,1)
ax[0].plot(x[i,0,:,0])
ax[0].set_ylim([-5,5])
ax[0].set_xlim([0,200])
ax[0].set_xticks([])
ax[0].set_yticks([])    
# fig.savefig("orig.svg",format="svg")

# Plot augmented example signal
t = np.linspace(0,0.2,200)
# for amp in range(1,6):
#     # random noise
#     # aug_n = np.random.normal(0,amp,200)
#     # 60 hz noise
#     aug_n = amp*np.sin(2*np.pi*60*t)
#     aug = prd.truncate(x[i,0,:,0] + aug_n)
    
#     fig,ax = plt.subplots(2,1)
#     # ax[0].plot(x[i,0,:,0])
#     ax[0].plot(aug_n)
#     ax[1].plot(aug)
#     for j in range(2):
#         ax[j].set_ylim([-5,5])
#         ax[j].set_xlim([0,200])
#         ax[j].set_xticks([])
#         ax[j].set_yticks([])

    # save fig
    # fig.savefig("sin" + str(amp) + ".svg", format="svg")
fig,ax = plt.subplots(6,3)
x_aug = cp.deepcopy(x)
aug_n = 2*np.sin(2*np.pi*60*t)
x_aug[i,1,:,0] = prd.truncate(x[i,1,:,0] + aug_n)
aug_n = np.random.normal(0,3,200)
x_aug[i,5,:,0] = prd.truncate(x[i,5,:,0] + aug_n)
x_aug[i,4,:,0] = 0
x_real = cp.deepcopy(x)
real_n = real_noise_temp[0,370,:]
x_real[i,3,:,0] = prd.truncate(x[i,3,:,0] + emgscale[3]*real_n)
real_n = real_noise_temp[2,370,:]
x_real[i,4,:,0] = prd.truncate(x[i,4,:,0] + emgscale[4]*real_n)
real_n = real_noise_temp[4,150,:]
x_real[i,0,:,0] = prd.truncate(x[i,0,:,0] + emgscale[0]*real_n)
for ch in range(6):
    ax[ch,0].plot(x[i,ch,:,0])
    ax[ch,1].plot(x_aug[i,ch,:,0])#-x[i,ch,:,0])
    ax[ch,2].plot(x_real[i,ch,:,0])#-x[i,ch,:,0])
    for j in range(3):
        ax[ch,j].set_ylim([-5,5])
        ax[ch,j].set_xlim([0,200])
    
        if ch > 0:
            ax[ch,j].set_yticks([])
        if ch < 5:
            ax[ch,j].set_xticks([])

fig.tight_layout

Loading training data: traindata_all/TR1_traindata_2.p


<bound method Figure.tight_layout of <Figure size 640x480 with 18 Axes>>

In [9]:
plt.plot(real_noise_temp[0,370,:])

Timing

In [3]:
def model_train(d, model, optimizer, train_loss, train_accuracy, x_train_in, p_train, noise): 
    if isinstance(model,CNN):
        mod = 'cnn'
    else:
        mod = 'mlp'   
    ds, y_train, x_train = prd.prep_train_data_lite(d, x_train_in, p_train, mod, noise)
    train_mod = get_train()

    for epoch in range(30):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y in ds:
            train_mod(x, y, model, optimizer, train_loss, train_accuracy)

    y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]
    temp = model.enc(x_train[:x_train.shape[0]//2,...]).numpy()
    temp2 = np.vstack((temp,model.enc(x_train[x_train.shape[0]//2:,...]).numpy()))
    w, c,_, _, _ = train_lda(temp2,y_train_aug)
    return model, w, c, x_train
    
def pred(model, w, c, x):
    data = model.enc(x).numpy()

    f = np.dot(w,data.T) + c
    return np.argmax(f, axis=0)

In [5]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven14'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = np.max(params[:,0])
data_load = False
test = False
train = True

sub = 1
print('Running ' + str(sess.sub_type) + str(sub))
sess.sub = sub
tf.keras.backend.clear_session()

train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

if np.sum(train_ind) and np.sum(test_ind):
    n_dof = np.max(params[train_ind,4])

    # Train NNs
    mlp = MLP(n_class=n_dof)
    cnn = CNN(n_class=n_dof)

    optimizer = tf.keras.optimizers.Adam()
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

    # Train aligned LDA
    x_train, _, _, p_train, _, _ = prd.train_data_split(raw,params,sess.sub,sess.sub_type,dt=sess.cv_type,load=True,train_grp=sess.train_grp)
    
    cnn, w, c, x = model_train(sess, cnn, optimizer, train_loss, train_accuracy, x_train, p_train, noise=False)

    %timeit pred(cnn,w,c, x[:1,...])

    # %timeit model_train(sess, cnn, optimizer, train_loss, train_accuracy, x_train, p_train, noise=False)

    # del mlp, cnn
    # mlp = MLP(n_class=n_dof)
    # cnn = CNN(n_class=n_dof)

    # %timeit model_train(sess, mlp, optimizer, train_loss, train_accuracy, x_train, p_train, noise=True)

    # %timeit model_train(sess, cnn, optimizer, train_loss, train_accuracy, x_train, p_train, noise=True)
  

Running TR1
Loading training data: traindata_all/TR1_traindata_2.p
7.75 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
